# Zenith Conda Package Build (with CUDA)

This notebook builds a Conda package with CUDA kernels for distribution.

**Prerequisites:**
- GPU Runtime enabled (Runtime > Change runtime type > T4 GPU)
- Anaconda Cloud account (https://anaconda.org)

In [ ]:
# Cell 1: Check GPU and CUDA
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv
!nvcc --version

In [ ]:
# Cell 2: Install Miniconda
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /content/miniconda
import os
os.environ['PATH'] = '/content/miniconda/bin:' + os.environ['PATH']
print("Miniconda installed!")

In [ ]:
# Cell 3: Accept Terms of Service and install conda-build
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main -y || true
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r -y || true
!conda install -y conda-build anaconda-client -c conda-forge
print("\nconda-build installed!")
!conda build --version

In [ ]:
# Cell 4: Clone Zenith repository
!rm -rf ZENITH
!git clone https://github.com/vibeswithkk/ZENITH.git
%cd ZENITH
!git log -1 --oneline

In [ ]:
# Cell 5: Create conda recipe (meta.yaml)
import os
os.makedirs('conda', exist_ok=True)

meta_yaml = '''package:
  name: pyzenith-cuda
  version: "0.2.0"

source:
  path: ..

build:
  number: 0
  script: pip install .
  
requirements:
  host:
    - python >=3.9
    - pip
    - numpy
  run:
    - python >=3.9
    - numpy >=1.20

test:
  imports:
    - zenith
  commands:
    - python -c "import zenith; print(zenith.__version__)"

about:
  home: https://github.com/vibeswithkk/ZENITH
  license: MIT
  license_file: LICENSE
  summary: High-performance ML compiler framework with CUDA acceleration
'''

with open('conda/meta.yaml', 'w') as f:
    f.write(meta_yaml)

print("meta.yaml created!")
!cat conda/meta.yaml

In [ ]:
# Cell 6: Build Conda package
!conda build conda/ --output-folder /content/conda-output --no-anaconda-upload
print("\n" + "="*50)
print("BUILD COMPLETE!")
print("="*50)
!ls -la /content/conda-output/**/*.tar.bz2 2>/dev/null || echo "Checking output..."
!find /content/conda-output -name "*.tar.bz2" 2>/dev/null

In [ ]:
# Cell 7: Login to Anaconda Cloud
# Get token from: https://anaconda.org/settings/access
# Create new token with "Allow all API operations"

ANACONDA_TOKEN = "YOUR_TOKEN_HERE"  # <-- GANTI INI dengan token kamu

if ANACONDA_TOKEN != "YOUR_TOKEN_HERE":
    !anaconda -t $ANACONDA_TOKEN whoami
    print("Login successful!")
else:
    print("Please set your ANACONDA_TOKEN first!")
    print("\nSteps:")
    print("1. Go to: https://anaconda.org/settings/access")
    print("2. Click 'Add Token'")
    print("3. Name it 'colab-upload'")
    print("4. Check 'Allow all API operations'")
    print("5. Copy the token and paste above")

In [ ]:
# Cell 8: Upload to Anaconda Cloud OR Download
import glob

packages = glob.glob('/content/conda-output/**/*.tar.bz2', recursive=True)
print(f"Found {len(packages)} package(s):")
for pkg in packages:
    print(f"  - {pkg}")

if ANACONDA_TOKEN != "YOUR_TOKEN_HERE" and packages:
    print("\nUploading to Anaconda Cloud...")
    for pkg in packages:
        !anaconda -t $ANACONDA_TOKEN upload $pkg --force
    print("\nUpload complete!")
elif packages:
    print("\nDownloading package to your computer...")
    from google.colab import files
    for pkg in packages:
        files.download(pkg)
else:
    print("\nNo packages found. Check build output above for errors.")

## Done!

After upload, users can install with:
```bash
conda install -c YOUR_USERNAME pyzenith-cuda
```

Or if downloaded manually, install with:
```bash
conda install pyzenith-cuda-0.2.0-*.tar.bz2
```